In [10]:
import numpy as np
import pandas as pd

In [11]:
stock = "hs300"

In [12]:
pred = np.load(f"{stock}_output/wealth4.npy")

In [13]:
pred = pd.DataFrame(pred.reshape(-1, 1), columns=['daily_return'])
pred

,daily_return
0,1.000000
1,1.008859
2,1.027444
3,1.027611
4,1.025633
...,...
236,0.862062
237,0.871034
238,0.865776
239,0.864022


In [14]:
pred.to_csv(f"{stock}_ARR.csv", index=None)

In [15]:
pred['daily_return'] = pred['daily_return'] / pred['daily_return'].shift(1)
pred['daily_return'] = pred['daily_return'] - 1
pred.dropna(inplace=True)
pred

,daily_return
1,0.008859
2,0.018422
3,0.000162
4,-0.001925
5,0.000182
...,...
236,0.007665
237,0.010408
238,-0.006036
239,-0.002026


In [16]:
portfolio_df_performance = pred

In [17]:
alpha_df_performance = pd.DataFrame()
alpha_df_performance['portfolio_daily_return'] = portfolio_df_performance['daily_return']
alpha_df_performance['portfolio_net_value'] = (alpha_df_performance['portfolio_daily_return'] + 1).cumprod()

net_value_columns = ['portfolio_net_value']

alpha_statistics_df = pd.DataFrame(index=alpha_df_performance[net_value_columns].columns,
                                    columns=["年化收益", "年化波动率", "最大回撤率", "夏普率", "Calmar", "IR"])

# alpha_df_performance.set_index("dt", inplace=True)
alpha_df_performance.index = pd.to_datetime(alpha_df_performance.index)
monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
monthly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    monthly_statistics_df])
monthly_statistics_df = monthly_statistics_df.pct_change()
monthly_statistics_df = monthly_statistics_df.dropna()
monthly_statistics_df.index = monthly_statistics_df.index.date
## TODO 补充第一年的数据
yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
yearly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    yearly_statistics_df])
yearly_statistics_df = yearly_statistics_df.pct_change()
yearly_statistics_df = yearly_statistics_df.dropna()
yearly_statistics_df.index = yearly_statistics_df.index.date

alpha_statistics_df.loc[:, "年化收益"] = np.mean(
    (alpha_df_performance[net_value_columns].tail(1)) ** (252 / len(alpha_df_performance)) - 1)
alpha_statistics_df.loc[:, "年化波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252)
alpha_statistics_df.loc[:, "累积收益"] = np.mean(alpha_df_performance[net_value_columns].tail(1) - 1)
alpha_statistics_df.loc[:, "累积波动率"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)
alpha_statistics_df.loc[:, "最大回撤率"] = np.min(
    (alpha_df_performance[net_value_columns] - alpha_df_performance[net_value_columns].cummax()) /
    alpha_df_performance[net_value_columns].cummax())
alpha_statistics_df.loc[:, "夏普率"] = alpha_statistics_df["年化收益"] / alpha_statistics_df["年化波动率"]
alpha_statistics_df.loc[:, "Calmar"] = alpha_statistics_df["年化收益"] / abs(alpha_statistics_df["最大回撤率"])
alpha_statistics_df.loc[:, "IR"] = np.mean(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252) / np.std(alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)

C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_7844\980728215.py:12: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
C:\Users\胡逸凡\AppData\Local\Temp\ipykernel_7844\980728215.py:20: FutureWarning: 'y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
C:\Users\胡逸凡\AppData\Roaming\Python\Python310\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame

In [18]:
alpha_statistics_df

,年化收益,年化波动率,最大回撤率,夏普率,Calmar,IR,累积收益,累积波动率
portfolio_net_value,-0.121783,0.147073,-0.228532,-0.828046,-0.532894,-0.876184,-0.116336,0.009265


Deeptrader
ASR = ARR / AVol;   CR = ARR / abs(MDD)
        ARR    AVol    MDD    ASR    CR    IR
hs300 -0.122  0.147  -0.229 -0.828 -0.533 -0.876

zz500  0.055  0.168  -0.141  0.324  0.388  0.370

sp500  0.295  0.180  -0.181  1.635  1.628  1.425
   
nas100 0.716  0.248  -0.138  2.890  5.169  2.306